## Домашнее задание

### 1. Напишите текстом как минимум 6 функций распределения и соответствующие им функции правдоподобия. Напишите, какие по вашему мнению целевые переменные из бизнеса (лучше - из страхового) соответствуют этим распределениям (по 2 примера на каждое). Пример: биномиальное распределение для задачи моделирования пролонгации (продлил клиент полис на следующий период или нет).

Логнормальное распределение - убытки в автостраховании и страховании здоровья (сильные влияния больших значений и сильная ассимитрия в сторону малых значений)

Плотность распределения: $$P(x) = \frac{1}{x\sigma\sqrt{2\pi}}e^{-\frac{(lnx-m)^2}{2\sigma^2}}$$

Функция правдоподобия: $$L(x) = \frac{1}{(2\pi \sigma^2)^\frac{n}{2}}e^{-\frac{\sum(lnx-m)^2}{2\sigma^2}}$$

Обобщенное распределение Парето - распределение страховых убытков с "Тяжелыми хвостами" (много убытков от небольшого количества страховых случаев, по которым выплаты большие)

Плотность распределения: $$P(x) = \frac{\alpha}{x_0}(\frac{x_0}{x})^{1+\alpha}$$

Функция правдоподобия: $$L(x) = (\frac{\alpha}{x_0})^n(\frac{x_0^{n(1+\alpha)}}{\prod_{i=1}^{n}x_i^{1+\alpha}})$$

Распределение Вейбулла - убытки в страховании, если страховой риск связан с отказом какого-либо технического устройства (отказ бытовой техники или поломка автомобиля)

Функция распределения: $$F(x) = 1-e^{-(\frac{x}{\lambda})^k} $$

Функция правдоподобия: $$L(x) = \prod_{i=1}^{n} (1-e^{-(\frac{x_i}{\lambda})^k}) $$

Гамма распределение - значения страховых убытков располагаются несимметрично относительно математического ожидания  - убытки в автостраховании и страховании здоровья 

Функция распределения: $$F(x) = \frac{1}{\Gamma(\lambda)} \int_{0}^{x} y^{\lambda-1}e^{-y}dy $$

Функция правдоподобия: $$L(x) = \prod_{i=1}^{n} \frac{1}{\Gamma(\lambda)} \int_{0}^{x_i} y^{\lambda-1}e^{-y}dy $$

Распределение Бернулли - страхование на случай смерти, тотальный урон автомобилю

Функция распределения: $$F(x) = 0, x<0;  q, 0<=x<1; 1, x>=1 $$

Функция правдоподобия: $$L(x) = \prod_{i=1}^{n} x_ilnp+(1-x_i)ln(1-p)$$

Распределение Пуассона - распределение числа страховых случаев для каждого конкретного застрахованного автомобиля в течение года, распределение числа страховых случаев (травм) для каждого конкретного застрахованного человека в течение года

Плотность распределения: $$P(x) = \frac{e^{-\lambda}{\lambda^x}}{x!} $$

Функция правдоподобия: $$L(x) = e^{-\lambda n} \prod_{i=1}^{n} \frac{\lambda^x_i}{x_i!} $$

### 2. На основе датасета из ноутбука проведите анализ данных и постройте зависимости частоты и среднего убытка от какого-либо параметра (за исключением уже выполненных в ноутбуке).

In [3]:
# Загрузка библиотек

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
# Загрузка набора данных в pandas DataFrame

df = pd.read_csv('freMPL-R.csv', low_memory=False)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343080 entries, 0 to 343079
Data columns (total 31 columns):
Exposure             343080 non-null float64
LicAge               343080 non-null int64
RecordBeg            343080 non-null object
RecordEnd            181115 non-null object
VehAge               177880 non-null object
Gender               343080 non-null object
MariStat             343080 non-null object
SocioCateg           343080 non-null object
VehUsage             343080 non-null object
DrivAge              343080 non-null int64
HasKmLimit           343080 non-null int64
BonusMalus           343080 non-null int64
VehBody              145780 non-null object
VehPrice             145780 non-null object
VehEngine            145780 non-null object
VehEnergy            145780 non-null object
VehMaxSpeed          145780 non-null object
VehClass             145780 non-null object
ClaimAmount          343080 non-null float64
RiskVar              145780 non-null float64
Garage    

Поскольку freMPL состоит из 10 отдельных датасетов, каждый из которых имеет различные переменные, есть смысл заранее посмотреть, какие датасеты стоит использовать, чтобы затем не возникало проблем с большим количеством пропущенных данных.

In [6]:
# Смотрим, какие факторы пропущены в каждом из датасетов

dct = {}
for i in range(1,11):
    _x = df.loc[df.Dataset == i].notnull().sum()
    dct[i] = list(_x[_x == 0].index)

print('Dataset  Missing Variables')
for x in range(1,11):
    print(x,'\t',dct[x])

Dataset  Missing Variables
1 	 ['DeducType', 'ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen', 'OutUseNb', 'RiskArea']
2 	 ['DeducType', 'ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen', 'OutUseNb', 'RiskArea']
3 	 ['ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen', 'OutUseNb', 'RiskArea']
4 	 ['ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen', 'OutUseNb', 'RiskArea']
5 	 ['VehAge', 'VehBody', 'VehPrice', 'VehEngine', 'VehEnergy', 'VehMaxSpeed', 'VehClass', 'RiskVar', 'Garage', 'DeducType']
6 	 ['VehAge', 'VehBody', 'VehPrice', 'VehEngine', 'VehEnergy', 'VehMaxSpeed', 'VehClass', 'RiskVar', 'Garage', 'DeducType']
7 	 ['VehAge', 'VehBody', 'VehPrice', 'VehEngine', 'VehEnergy', 'VehMaxSpeed', 'VehClass', 'RiskVar', 'Garage', 'DeducType']
8 	 ['VehAge', 'VehBody', 'VehPrice', 'VehEngine', 'VehEnergy', 'VehMaxSpeed', 'VehClass

Наборы данных 1-4 не содержат информации о количестве страховых требований, а 5-10 не содержат информации по характеристикам транспортного средства. Тем не менее, наборы 5-9 имеют одинаковые пропущенные факторы, имеет смысл попробовать их объединить.

In [7]:
# Объединяем наборы данных 5-9, удаляем пустые столбцы, удаляем дубликаты

df59 = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df59 = df59.drop(['Dataset'], axis=1)
df59 = df59.dropna(axis=1, how='all')
print('With duplicates\t\t', len(df59))
df59 = df59.drop_duplicates()
print('Without duplicates\t', len(df59))

With duplicates		 165200
Without duplicates	 115155


In [8]:
df = df59

In [9]:
# Вспомогательный столбец для суммирования числа полисов

df['PolicyCount'] = 1

In [10]:
# Вспомогательный столбец для суммирования числа полисов с убытками

df['ClaimCount'] = df['ClaimAmount'] > 0

In [11]:
# Вспомогательный столбец для суммирования числа полисов без убытков

df['NoClaimCount'] = df.PolicyCount - df.ClaimCount

In [12]:
df[['PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].head()

,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount
145780,1,0.0,False,1
145781,1,0.0,False,1
145782,1,0.0,False,1
145783,1,0.0,False,1
145784,1,0.0,False,1


### На занятии были исследованы зависимости от следующих признаков DrivAge, LicAge, Gender, Gender_Age, MariStat, Gender_MariStat_Age, SocioCateg. Продолжим:

#### Зависимость от BonusMalus

In [15]:
df_group_BM = df[['Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].groupby(df['BonusMalus']).sum()
df_group_BM = df_group_BM.reset_index()

In [19]:
df_group_BM.head()

,BonusMalus,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount
0,50,30659.216,67994,1.112703e+07,5831.0,62163
1,51,953.531,2218,3.017447e+05,184.0,2034
2,52,386.436,915,2.110474e+05,83.0,832
3,53,240.039,551,1.251411e+05,64.0,487
4,54,1030.683,2413,3.602272e+05,225.0,2188


In [20]:
# Freq - число страховок, по которым проводились выплаты, по отношению к экспозиции
df_group_BM['Freq'] = df_group_BM.ClaimCount / df_group_BM.Exposure
# AvgClaim - средняя сумма выплаты по страховке
df_group_BM['AvgClaim'] = df_group_BM.ClaimAmount / df_group_BM.ClaimCount
# ClaimRate - доля страховок, по которым производились выплаты к общему числу страховок
df_group_BM['ClaimRate'] = df_group_BM.ClaimCount/df_group_BM.PolicyCount

In [21]:
df_group_BM

,BonusMalus,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount,Freq,AvgClaim,ClaimRate
0,50,30659.216,67994,1.112703e+07,5831.0,62163,0.190188,1908.253563,0.085758
1,51,953.531,2218,3.017447e+05,184.0,2034,0.192967,1639.916702,0.082958
2,52,386.436,915,2.110474e+05,83.0,832,0.214783,2542.739242,0.090710
3,53,240.039,551,1.251411e+05,64.0,487,0.266623,1955.329362,0.116152
4,54,1030.683,2413,3.602272e+05,225.0,2188,0.218302,1601.009934,0.093245
...,...,...,...,...,...,...,...,...,...
99,175,1.766,6,1.136538e+03,1.0,5,0.566251,1136.537764,0.166667
100,178,0.636,1,0.000000e+00,0.0,1,0.000000,NaN,0.000000
101,183,1.665,2,3.339154e+02,1.0,1,0.600601,333.915408,0.500000
102,185,1.582,3,0.000000e+00,0.0,3,0.000000,NaN,0.000000


In [24]:
fig = px.bar(df_group_BM, x='BonusMalus', y='Freq', title='Зависимость частоты убытков от КБМ')
fig.show()

In [25]:
fig = px.bar(df_group_BM, x='BonusMalus', y='ClaimRate', title='Зависимость доли убыточных страховок от КБМ')
fig.show()

In [26]:
fig = px.bar(df_group_BM, x='BonusMalus', y='AvgClaim', title='Зависимость среднего убытка от КБМ')
fig.show()

#### Зависимость от VehUsage

In [27]:
df.columns

Index(['Exposure', 'LicAge', 'RecordBeg', 'RecordEnd', 'Gender', 'MariStat',
       'SocioCateg', 'VehUsage', 'DrivAge', 'HasKmLimit', 'BonusMalus',
       'ClaimAmount', 'ClaimInd', 'ClaimNbResp', 'ClaimNbNonResp',
       'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen', 'OutUseNb',
       'RiskArea', 'PolicyCount', 'ClaimCount', 'NoClaimCount'],
      dtype='object')

In [29]:
df.VehUsage.value_counts()

Private+trip to office    59834
Private                   38839
Professional              14302
Professional run           2180
Name: VehUsage, dtype: int64

In [32]:
df_group_VU = df[['Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].groupby(df['VehUsage']).sum()
df_group_VU = df_group_VU.reset_index()

In [34]:
# Freq - число страховок, по которым проводились выплаты, по отношению к экспозиции
df_group_VU['Freq'] = df_group_VU.ClaimCount / df_group_VU.Exposure
# AvgClaim - средняя сумма выплаты по страховке
df_group_VU['AvgClaim'] = df_group_VU.ClaimAmount / df_group_VU.ClaimCount
# ClaimRate - доля страховок, по которым производились выплаты к общему числу страховок
df_group_VU['ClaimRate'] = df_group_VU.ClaimCount/df_group_VU.PolicyCount

In [35]:
df_group_VU

,VehUsage,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount,Freq,AvgClaim,ClaimRate
0,Private,17385.293,38839,5.630104e+06,3066.0,35773,0.176356,1836.302806,0.078941
1,Private+trip to office,25945.001,59834,1.232871e+07,5812.0,54022,0.224012,2121.250126,0.097135
2,Professional,6574.757,14302,3.658391e+06,1718.0,12584,0.261302,2129.447356,0.120123
3,Professional run,977.100,2180,4.318442e+05,273.0,1907,0.279398,1581.846871,0.125229


In [36]:
fig = px.bar(df_group_VU, x='VehUsage', y='Freq', title='Зависимость частоты убытков от VehUsage')
fig.show()

In [37]:
fig = px.bar(df_group_VU, x='VehUsage', y='ClaimRate', title='Зависимость доли убыточных страховок от VehUsage')
fig.show()

In [39]:
fig = px.bar(df_group_VU, x='VehUsage', y='AvgClaim', title='Зависимость среднего убытка от VehUsage')
fig.show()

#### Зависимость от RiskArea

In [41]:
df_group_RA = df[['Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].groupby(df['RiskArea']).sum()
df_group_RA = df_group_RA.reset_index()

In [43]:
# Freq - число страховок, по которым проводились выплаты, по отношению к экспозиции
df_group_RA['Freq'] = df_group_RA.ClaimCount / df_group_BM.Exposure
# AvgClaim - средняя сумма выплаты по страховке
df_group_RA['AvgClaim'] = df_group_RA.ClaimAmount / df_group_BM.ClaimCount
# ClaimRate - доля страховок, по которым производились выплаты к общему числу страховок
df_group_RA['ClaimRate'] = df_group_RA.ClaimCount/df_group_BM.PolicyCount

In [44]:
df_group_RA.head()

,RiskArea,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount,Freq,AvgClaim,ClaimRate
0,1.0,42.687,101,8.086774e+03,5.0,96,0.000163,1.386859,0.000074
1,2.0,701.934,1610,1.981481e+05,143.0,1467,0.149969,1076.891918,0.064472
2,3.0,878.777,1979,3.362095e+05,181.0,1798,0.468383,4050.716766,0.197814
3,4.0,1703.435,3931,6.589674e+05,428.0,3503,1.783044,10296.365257,0.776770
4,5.0,3125.233,7112,1.187095e+06,715.0,6397,0.693715,5275.979698,0.296312


In [45]:
fig = px.bar(df_group_RA, x='RiskArea', y='Freq', title='Зависимость частоты убытков от RiskArea')
fig.show()

In [46]:
fig = px.bar(df_group_RA, x='RiskArea', y='ClaimRate', title='Зависимость доли убыточных страховок от RiskArea')
fig.show()

In [47]:
fig = px.bar(df_group_RA, x='RiskArea', y='AvgClaim', title='Зависимость среднего убытка от RiskArea')
fig.show()

#### Зависимость от OutUseNb

In [54]:
df_group_OUN = df[['Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].groupby(df['OutUseNb']).sum()
df_group_OUN = df_group_OUN.reset_index()

In [56]:
# Freq - число страховок, по которым проводились выплаты, по отношению к экспозиции
df_group_OUN['Freq'] = df_group_OUN.ClaimCount / df_group_OUN.Exposure
# AvgClaim - средняя сумма выплаты по страховке
df_group_OUN['AvgClaim'] = df_group_OUN.ClaimAmount / df_group_OUN.ClaimCount
# ClaimRate - доля страховок, по которым производились выплаты к общему числу страховок
df_group_OUN['ClaimRate'] = df_group_OUN.ClaimCount/df_group_OUN.PolicyCount

In [57]:
df_group_OUN.head()

,OutUseNb,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount,Freq,AvgClaim,ClaimRate
0,0.0,43011.034,96541,1.777945e+07,8783.0,87758,0.204203,2024.302220,0.090977
1,1.0,5162.812,12027,2.470638e+06,1318.0,10709,0.255287,1874.535628,0.109587
2,2.0,1565.959,3747,1.023859e+06,435.0,3312,0.277785,2353.699852,0.116093
3,3.0,716.660,1765,3.866439e+05,199.0,1566,0.277677,1942.934125,0.112748
4,4.0,295.399,748,3.074217e+05,100.0,648,0.338525,3074.216828,0.133690


In [59]:
fig = px.bar(df_group_OUN, x='OutUseNb', y='Freq', title='Зависимость частоты убытков от OutUseNb')
fig.show()

In [60]:
fig = px.bar(df_group_OUN, x='OutUseNb', y='ClaimRate', title='Зависимость доли убыточных страховок от OutUseNb')
fig.show()

In [61]:
fig = px.bar(df_group_OUN, x='OutUseNb', y='AvgClaim', title='Зависимость среднего убытка от OutUseNb')
fig.show()

#### Зависимость от HasKmLimit

In [62]:
df_group_HKL = df[['Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].groupby(df['HasKmLimit']).sum()
df_group_HKL = df_group_HKL.reset_index()

In [63]:
# Freq - число страховок, по которым проводились выплаты, по отношению к экспозиции
df_group_HKL['Freq'] = df_group_HKL.ClaimCount / df_group_OUN.Exposure
# AvgClaim - средняя сумма выплаты по страховке
df_group_HKL['AvgClaim'] = df_group_HKL.ClaimAmount / df_group_OUN.ClaimCount
# ClaimRate - доля страховок, по которым производились выплаты к общему числу страховок
df_group_HKL['ClaimRate'] = df_group_HKL.ClaimCount/df_group_OUN.PolicyCount

In [64]:
df_group_HKL.head()

,HasKmLimit,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount,Freq,AvgClaim,ClaimRate
0,0,45342.629,102495,2.066682e+07,10093.0,92402,0.234661,2353.047387,0.104546
1,1,5539.522,12660,1.382230e+06,776.0,11884,0.150306,1048.732696,0.064521


In [65]:
fig = px.bar(df_group_HKL, x='HasKmLimit', y='Freq', title='Зависимость частоты убытков от HasKmLimit')
fig.show()

In [67]:
fig = px.bar(df_group_HKL, x='HasKmLimit', y='ClaimRate', title='Зависимость доли убыточных страховок от HasKmLimit')
fig.show()

In [68]:
fig = px.bar(df_group_HKL, x='HasKmLimit', y='AvgClaim', title='Зависимость среднего убытка от HasKmLimit')
fig.show()